In [37]:
import torch
import pickle
import numpy as np
from torch.nn import functional as F
from torch import nn 
import numpy as np
import wandb
import tqdm
import d2l.torch as d2l
import math

In [38]:
def make_model(classes):
    filter_num = ['None',32,64,128,256]
    kernel_size = ['None',9,9,9,9]
    conv_stride_size = ['None',1,1,1,1]
    pool_stride_size = ['None',4,4,4,4]
    pool_size = ['None',8,8,8,8]
    net = nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=filter_num[1], kernel_size=kernel_size[1], stride=conv_stride_size[1], padding=4),
        nn.BatchNorm1d(num_features=32),
        nn.ELU(alpha=1.0),
        nn.Conv1d(in_channels=32, out_channels=filter_num[1], kernel_size=kernel_size[1], stride=conv_stride_size[1], padding=4),
        nn.BatchNorm1d(num_features=32),
        nn.ELU(alpha=1.0),
        nn.MaxPool1d(kernel_size=pool_size[1], stride=pool_stride_size[1], padding=2),
        nn.Dropout(p=0.1),
        nn.Conv1d(in_channels=32, out_channels=filter_num[2], kernel_size=kernel_size[2], stride=conv_stride_size[2], padding=4),
        nn.BatchNorm1d(num_features=64),
        nn.ReLU(),
        nn.Conv1d(in_channels=64, out_channels=filter_num[2], kernel_size=kernel_size[2], stride=conv_stride_size[2], padding=4),
        nn.BatchNorm1d(num_features=64),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=pool_size[2], stride=pool_stride_size[2], padding=3),
        nn.Dropout(p=0.1),
        nn.Conv1d(in_channels=64, out_channels=filter_num[3], kernel_size=kernel_size[3], stride=conv_stride_size[3], padding=4),
        nn.BatchNorm1d(num_features=128),
        nn.ReLU(),
        nn.Conv1d(in_channels=128, out_channels=filter_num[3], kernel_size=kernel_size[3], stride=conv_stride_size[3], padding=4),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=pool_size[3], stride=pool_stride_size[3], padding=4),
        nn.Dropout(p=0.1),
        nn.Conv1d(in_channels=128, out_channels=filter_num[4], kernel_size=kernel_size[4], stride=conv_stride_size[4], padding=4),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Conv1d(in_channels=256, out_channels=filter_num[4], kernel_size=kernel_size[4], stride=conv_stride_size[4], padding=4),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=pool_size[4], stride=pool_stride_size[4], padding=4),
        nn.Dropout(p=0.1),
        nn.Flatten(start_dim=-2),
        nn.Linear(256 * 20, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=0.7),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(512, classes)
    )
    return net

In [39]:
device = torch.device(f'cuda:{6}')
net = make_model(96)
state_dict = torch.load('/home/xjj/projects/graduate_project/df_wtf_pad/wtf_pad_ow_model')
print ("Model loaded!")
print ("Use the model from: ", '/home/xjj/projects/graduate_project/df_wtf_pad/wtf_pad_ow_model')
net.load_state_dict(state_dict)
net.to(device)

Model loaded!
Use the model from:  /home/xjj/projects/graduate_project/df_wtf_pad/wtf_pad_ow_model


Sequential(
  (0): Conv1d(1, 32, kernel_size=(9,), stride=(1,), padding=(4,))
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ELU(alpha=1.0)
  (3): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,))
  (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ELU(alpha=1.0)
  (6): MaxPool1d(kernel_size=8, stride=4, padding=2, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.1, inplace=False)
  (8): Conv1d(32, 64, kernel_size=(9,), stride=(1,), padding=(4,))
  (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,))
  (12): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): MaxPool1d(kernel_size=8, stride=4, padding=3, dilation=1, ceil_mode=False)
  (15): Dropout(p=0.1, inplace=False)
  (16): Conv1d(64, 128, kernel_size=(9,), stride=(1,), pa

In [40]:
def LoadDataWTFPADOW_Evaluation():

    print ("Loading WTFPAD dataset for open-world scenario for evaluation")
    # Point to the directory storing data
    dataset_dir = '/data/Deep_fingerprint/wtf_pad/open_world/'

    # X represents a sequence of traffic directions
    # y represents a sequence of corresponding label (website's label)

    # Load training data
    with open(dataset_dir + 'X_test_Mon_WTFPAD.pkl', 'rb') as handle:
        X_test_Mon = pickle.load(handle, encoding='latin1')
    with open(dataset_dir + 'y_test_Mon_WTFPAD.pkl', 'rb') as handle:
        y_test_Mon = pickle.load(handle, encoding='latin1')
    with open(dataset_dir + 'X_test_Unmon_WTFPAD.pkl', 'rb') as handle:
        X_test_Unmon = pickle.load(handle, encoding='latin1')
    with open(dataset_dir + 'y_test_Unmon_WTFPAD.pkl', 'rb') as handle:
        y_test_Unmon = pickle.load(handle, encoding='latin1')

    X_test_Mon = np.array(X_test_Mon)
    y_test_Mon = np.array(y_test_Mon)
    X_test_Unmon = np.array(X_test_Unmon)
    y_test_Unmon = np.array(y_test_Unmon)
    
    X_test_Mon = X_test_Mon[:, np.newaxis, :]
    X_test_Unmon = X_test_Unmon[:, np.newaxis, :]

    X_test_Mon = torch.FloatTensor(X_test_Mon)
    y_test_Mon = torch.FloatTensor(y_test_Mon)
    X_test_Unmon = torch.FloatTensor(X_test_Unmon)
    y_test_Unmon = torch.FloatTensor(y_test_Unmon)

    return X_test_Mon, y_test_Mon, X_test_Unmon, y_test_Unmon

In [41]:
def predict(dataset, model):
    model.eval()
    X_test_Mon = dataset['X_test_Mon']
    X_test_Unmon = dataset['X_test_Unmon']
    print ("Total testing data ", len(X_test_Mon) + len(X_test_Unmon))
     # 创建 DataLoader
    loader_Mon = torch.utils.data.DataLoader(X_test_Mon, batch_size=256, shuffle=False)
    loader_Unmon = torch.utils.data.DataLoader(X_test_Unmon, batch_size=256, shuffle=False)

    # 分批次处理输入数据，并将结果合并
    with torch.no_grad():
        result_Mon = torch.cat([F.softmax(model(x.to(device)), dim=1) for x in loader_Mon], dim=0)
        result_Unmon = torch.cat([F.softmax(model(x.to(device)), dim=1) for x in loader_Unmon], dim=0)
    
    return result_Mon, result_Unmon

In [42]:
def evaluate(threshold_val, mon_label, unmon_label, result_mon, result_unmon, log_file):
    # Calculate the number of misclassifications
    mon_label = mon_label.cpu().numpy()
    unmon_label = unmon_label.cpu().numpy()
    result_mon = result_mon.cpu().numpy()
    result_unmon = result_unmon.cpu().numpy()

    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    # 要求预测类别正确
    mon_correct = (result_mon.argmax(axis=1)==mon_label).sum()
    unmon_correct = (result_unmon.argmax(axis=1)==unmon_label).sum()
    test_acc = (mon_correct + unmon_correct)/(len(mon_label)+len(unmon_label))    
    
    
    #置信度是对预测为正例的概率的要求
    for i in range(len(result_mon)):
        sm_vector = result_mon[i]
        predicted_class = np.argmax(sm_vector)
        max_prob = max(sm_vector)

        if predicted_class in mon_label: # predicted as Monitored
            if max_prob >= threshold_val: # predicted as Monitored and actual site is Monitored
                TP = TP + 1
            else: # predicted as Unmonitored and actual site is Monitored
                FN = FN + 1
        elif predicted_class in unmon_label: # predicted as Unmonitored and actual site is Monitored
            FN = FN + 1

    # ==============================================================
    # Test with Unmonitored testing instances
    # evaluation
    for i in range(len(result_unmon)):
        sm_vector = result_unmon[i]
        predicted_class = np.argmax(sm_vector)
        max_prob = max(sm_vector)

        if predicted_class in mon_label: # predicted as Monitored
            if max_prob >= threshold_val: # predicted as Monitored and actual site is Unmonitored
                FP = FP + 1
            else: # predicted as Unmonitored and actual site is Unmonitored
                TN = TN + 1
        elif predicted_class in unmon_label: # predicted as Unmonitored and actual site is Unmonitored
            TN = TN + 1

    
    print ("TP : ", TP)
    print ("FP : ", FP)
    print ("TN : ", TN)
    print ("FN : ", FN)
    print ("Total  : ", TP + FP + TN + FN)
    TPR = float(TP) / (TP + FN)
    print ("TPR : ", TPR)
    FPR = float(FP) / (FP + TN)
    print ("FPR : ",  FPR)
    Precision = float(TP) / (TP + FP)
    print ("Precision : ", Precision)
    Recall = float(TP) / (TP + FN)
    print ("Recall : ", Recall)
    log_file.writelines(f"two-*:\nthreshold_val: {threshold_val}\nTP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}\nTPR:{TPR}, FPR:{FPR}, Precision: {Precision}, Recall:{Recall}\n")
    
    log_file.writelines(f"test_acc:\nmon_correct: {mon_correct}  unmon_correct: {unmon_correct}\nmon_num: {len(mon_label)}   unmon_num: {len(unmon_label)}\nTest Accuracy: {test_acc}\n")
    

In [43]:
evaluation_type = 'OpenWorld_wtfpad'
print ("Evaluation type: ", evaluation_type)
threshold = 1.0 - 1 / np.logspace(0.05, 2, num=15, endpoint=True)
print('置信度: ', threshold)
file_name = f'/home/xjj/projects/graduate_project/df_wtf_pad/evaluation/{evaluation_type}_two.txt'
log_file =  open(file_name, "w")

Evaluation type:  OpenWorld_wtfpad
置信度:  [0.10874906 0.35328299 0.53072375 0.65947982 0.75290888 0.82070366
 0.86989748 0.90559391 0.93149626 0.95029174 0.96393027 0.97382678
 0.98100797 0.98621884 0.99      ]


In [44]:
dataset = {}
X_test_Mon, y_test_Mon, X_test_Unmon, y_test_Unmon = LoadDataWTFPADOW_Evaluation()

dataset['X_test_Mon'] = X_test_Mon
dataset['y_test_Mon'] = y_test_Mon
dataset['X_test_Unmon'] = X_test_Unmon
dataset['y_test_Unmon'] = y_test_Unmon

print ("Data loaded!")
print ("Loading DF model ...")
print ("The log file will be saved at ", file_name)
print ("-- The log file will contains")
print ("-- TP, FP, TN, FN, TPR, FPR, Precision, and Recall for each different threshold")
print ("-- These results will be used to plot the ROC or Precision&Recall Graph")

Loading WTFPAD dataset for open-world scenario for evaluation
Data loaded!
Loading DF model ...
The log file will be saved at  /home/xjj/projects/graduate_project/df_wtf_pad/evaluation/OpenWorld_wtfpad_two.txt
-- The log file will contains
-- TP, FP, TN, FN, TPR, FPR, Precision, and Recall for each different threshold
-- These results will be used to plot the ROC or Precision&Recall Graph


In [45]:
print(y_test_Mon.max(), y_test_Mon.min())     #94 0
print(y_test_Unmon.max(), y_test_Unmon.min())    #95 95

tensor(94.) tensor(0.)
tensor(95.) tensor(95.)


In [46]:
result_Mon, result_Unmon = predict(dataset, net)
print(result_Mon.shape, result_Unmon.shape)

Total testing data  29500
torch.Size([9500, 96]) torch.Size([20000, 96])


In [47]:
for th in threshold:
    evaluate(th, y_test_Mon, y_test_Unmon, result_Mon, result_Unmon, log_file)
log_file.close()

TP :  9203
FP :  5280
TN :  14720
FN :  297
Total  :  29500
TPR :  0.9687368421052631
FPR :  0.264
Precision :  0.6354346475177794
Recall :  0.9687368421052631
TP :  9071
FP :  4879
TN :  15121
FN :  429
Total  :  29500
TPR :  0.9548421052631579
FPR :  0.24395
Precision :  0.6502508960573476
Recall :  0.9548421052631579
TP :  8769
FP :  3801
TN :  16199
FN :  731
Total  :  29500
TPR :  0.9230526315789473
FPR :  0.19005
Precision :  0.6976133651551313
Recall :  0.9230526315789473
TP :  8499
FP :  2897
TN :  17103
FN :  1001
Total  :  29500
TPR :  0.8946315789473684
FPR :  0.14485
Precision :  0.7457879957879958
Recall :  0.8946315789473684
TP :  8269
FP :  2314
TN :  17686
FN :  1231
Total  :  29500
TPR :  0.870421052631579
FPR :  0.1157
Precision :  0.7813474440139847
Recall :  0.870421052631579
TP :  8101
FP :  1901
TN :  18099
FN :  1399
Total  :  29500
TPR :  0.8527368421052631
FPR :  0.09505
Precision :  0.8099380123975205
Recall :  0.8527368421052631
TP :  7941
FP :  1580
TN :  18